<a href="https://colab.research.google.com/github/phyop/220528_Colab/blob/main/220519_%E9%96%89%E5%8C%85_%E8%A3%9D%E9%A3%BE%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
《220519_閉包_裝飾器.ipynb》
https://www.udemy.com/course/python-en/learn/lecture/15690850#overview

# 裝飾器需要是利用閉包來實現

《類似變量有多組的話，就建立物件，用self.xx長期儲存屬性》

class Foo:
    # 方法中的變量值，會隨着方法調用的結束而消失。
    # 但如果是用self存儲的，也就是物件屬性，必須等到物件消失了，才會跟着消失。
    # __init__要有實例才會用得到，因此才加上self
    def __init__(self):
        pass

    # *和**代表不定長度，你傳也行，不傳也行，傳一個也行，傳多個也行
    def __call__(self, *args, **kwargs):
        print('__call__')


obj = Foo() # 執行__init__
# 自己沒有定義__call__沒關系，父類有就可以，父類沒有，那就一路往上找，找得到就可以call，找不到就報錯
# "物件名(引數)"，會去呼叫魔術方法__call__
obj() # 執行__call__
# 所以也就是說，雖然物件不是函數，但如果物件找得到__call__()，或是其父類以上有，就仍然可以視爲函數調用
# 換句話說，如果實例沒有__call__()方法，那就沒辦法調用「實例()」，比如數字


《「默認參數函數」太常被呼叫的話，乾脆就把函數提升成物件來使用》
# 雖然用字典給值(缺省參數、關鍵字參數)的方法也可以
def line_4(x, a=1, b=2):
    print(a*x+b)

# 但如果是10條y=ax+b的情況下，而且各條都要修改引數x的話，那就會變得很亂
line_4(0)
line_4(1)
line_4(2)
line_4(0, a=2, b=33)
line_4(1, a=2, b=33)
line_4(2, a=2, b=33)
line_4(0, a=3, b=44)
line_4(1, a=3, b=44)
line_4(2, a=3, b=44)
...


《物件最常被呼叫的方法，就設爲__call__》
class Line5(object):
    # 假設要創建10條y=ax+b，而且各條都要可以隨時改引數x的話
    # 可以利用創建物件的方式，讓每個物件都各帶有自己的屬性a、b，然後要改引數x的時候，再當作引數傳進去物件就好了
    def __init__(self, a, b):
        self.a = k
        self.b = b

    # 並且因爲最常用的就是做y=ax+b的計算，所以爲了減少還要寫方法名去呼叫計算函數，所以乾脆把要做的事，直接寫在__call__方法就好
    def __call__(self, x):
        print(self.a * x + self.b)

line_5_1 = Line5(1, 2)
line_5_1(0) # y=1*0+2
line_5_1(1) # y=1*1+2
line_5_1(2) # y=1*2+2
line_5_2 = Line5(11, 22)
line_5_1(0) # y=11*0+22
line_5_1(1) # y=11*1+22
line_5_1(2) # y=11*2+22


《閉包，外層是物件屬性a、b，內層是函數變量x，而且不需要多餘的繼承來消耗資源》

# 創建物件的話，繼承的object類會有原本有的各種魔法方法，那整體消耗的資源就會變多，所以Python就出現了閉包

def line_6(a, b):
    pass

# 在Python中，沒有return，那默認就是回傳None
line_6_1 = line_6(1, 2) # 在C裏面，這句話會報錯，因爲沒有return值


compiler走到「line_6_2 = line_6(11, 22)」的時候，line_6_2「變量指向一塊新創建的記憶體空間」，
裏面裝了內層函數create_y的內碼，並且(a, b) -> (11, 22)。
line_6_2(0)就是把x=0傳入到line_6_2的記憶體空間，也就是內層函數create_y的內碼。


閉包，要用到的其實是內層的函數，外部函數只是拿來創建物件，然後儲存屬性用的而已


# 1. compiler看到「def line_6」，就會略過，跳到「line_6_1 = line_6(1, 2)」
def line_6(a, b):
    # 5. 看到def，所以略過，直接跳到「return create_y」，然後就回傳create_y的地址，當作line_6(1, 2)的調用結果
    def create_y(x):
        print(a*x+b)
    return create_y 
    # create_y，指向函數「地址」的變量名
    # return create_y()，對指向的函數地址，做「函數調用」
    # line_6_1指向一個地址，也就是函數create_y的記憶體位置
    # 如果加上()，寫成return create_y()，那line_6_1指向的就是create_y()的返回值
    # 又因爲create_y()沒有寫return，所以default會返回None，也就是會讓line_6_1指向None去了
    # 加上()就是開始跑函數裏面的code，不加()就是我問問你在哪，你回給我地址就行了
    # 比如list、元組、字典就沒看過加()，因爲那就是一個地址，地址裏面只有值而已
    # 加()的有，比如寫上類的名字，那就是去跑__init__裏面的code；或寫上物件的名字，那就是去跑__call__裏面的code
    # 也就是，可以加()的，就代表裏面有code可以去調用出來跑


# 2. 然後因爲line_6()，所以會帶着實參(1, 2)去調用函數line_6
# 3. 也就是說，會在函數line_6下，創建一小塊記憶體空間，裏面保存着(a, b) -> (1, 2)實參
# 4. 這樣不就類比於函數line_6是類，一小塊記憶體空間是物件，實參(a, b) -> (1, 2)是物件屬性的意思了嗎！
# 6. 變量line_6_1指向create_y的地址，並且保存着(a, b) -> (1, 2)的實參
line_6_1 = line_6(1, 2) # 給予屬性值，創建物件


# 6. 所以，line_6_1(0) 就等於是，line_6_1是物件，裏面有物件屬性(a, b) -> (1, 2)，傳入實參x=0去調用物件方法create_y(x)
# 7. print出(a*x+b)的值爲2
# 8. 因爲"物件名(引數)"，會去呼叫魔術方法__call__，所以也可以看作成，__call__(x)的作用是去呼叫閉包的inner()
line_6_1(0) # 2 
line_6_1(1) # 3
line_6_1(2) # 4

line_6_2 = line_6(11, 22)
line_6_2(0) # 22
line_6_2(1) # 33
line_6_2(2) # 44


In [ ]:
《閉包 - 物件屬性(外層參數)、類屬性(外層def內，內層def外)》

global變數的缺點是，當程式量很大，有一堆函數的時候，明明只有幾個函數會用到，卻暴露讓所有函數都可以使用，
這也讓被hack的機會大大增加了，或是自己引用時不小心改到global的值。
所以使用object的好處就是，自己有關的變量、函數，封裝在自己的類裏面，既容易識別，又不容易引用錯誤，
一般的(變量, 函數)，就變成了object的(屬性, 方法)。


同理，閉包的固定屬性num、score，就相當於是類屬性，每個物件都具有一樣的值，
閉包的外層引數a、b，就相當於是物件屬性，每個物件都的值都不一樣，
閉包的內層函數，就相當於是物件方法，；閉包的內層引數x，就相當於是物件方法的形參。


def line_6(a, b): # 每個物件會有各自的參數值，所以類似「物件屬性」
    num = 100 # 每個物件都預先具有這個值，所以類似「類屬性」
    score = 200
    def create_y(x): # 物件方法(形參)
        print(a*x+b)
    return create_y

閉包和函數的差異是，閉包因爲回傳內層的函數名，所以可以呼叫內層函數的code，
還比函數多了物件屬性(外層的形參)、類屬性(定義在內層def外的屬性)。


# 匿名函數的優點是，可以直接當作實參，也就是可以直接填入匿名函數傳進去。
# foo指向一個(匿名)函數「地址」
foo = lambda x: x + 1
# 對指向的地址，進行「函數調用」
foo(3) # 4


In [ ]:
《裝飾器 - 局部變量》

def test1():
    x = 200 # 類屬性
    def test2(): # 物件方法
        x = 100 # 物件方法中的變量，是局部變量，所以不會修改到外部的x
        print("x=%d" %x) # 這邊會印出x=100
    return test2


x = 300
# 閉包就相當於class，所以名稱後面加上()，就相當於創建類的物件
# 閉包物件本身存有類屬性，以及創建時傳入的實參當作物件屬性
# 這邊相當於有類屬性x=200，並且因爲閉包test()沒有形參，所以相當於沒有物件屬性
t1 = test1()
# 對class的物件做調用，那就等同是調用__call__()
# 當對閉包物件做調用時，相當於帶入實參去調用inner()
# 因爲這邊的test2()沒有形參，所以t1()也不會帶入實參
t1() # x=100
# 定義局部變量，不會改變全局變量值
print(x) # 300
t1() # x=100

------------------

def test1():
    x = 200
    def test2():
        # 這層找不到x，就去上一層找
        print("x=%d" %x) # 這邊會印出x=200
    return test2


x = 300
# 建立閉包物件，類屬性x=200，物件屬性沒有，t1指向test2()
t1 = test1()
# 同名類屬性，並不會改變全局變量
print(x) # 300
# 調用閉包物件的inner()，所以執行print(x=...)
# 而在閉包物件中，找不到物件屬性x，所以改去類屬性那邊找看看，因此印出得到x=200
t1() # x=200
# 變量會優先在當層找，找不到才往上層找
# 而print(x)這條語句的層級是在全局，所以印出的是全局變量x的值
# 物件內並沒有對全局變量的x有做什麼事，所以全局變量的x維持值300
print(x) # 300

------------------


def test1():
    x = 200 # 類屬性
    def test2():
        # 這層有x，但是寫在這層的後面，所以報錯說
        # 我在這層是有找到局部變量x，但問題是，你都還沒到指派局部變量x，就硬要先呼叫引用，code的先後順序反了吧你？
        print("x=%d" %x)
        x = 100 # 物件方法中的局部變量
    return test2

t1 = test1()
t1() # UnboundLocalError: local variable 'x' referenced before assignment

------------------

def test1():
    x = 200 # 類屬性
    def test2():
        x = 100 # 物件方法中的局部變量
        # 因爲同層找得到x=100，所以不會去外層找類屬性的x=200
        print("x=%d" %x) # 這邊會印出局部變量x=100
    return test2


x = 300 # 全局變量
t1 = test1()
t1() # x=100
print(x) # 這邊會印出全局變量300

------------------

def test1():
    x = 200 # 類屬性
    def test2():
        # 同理，如果想要呼叫的是x=300的那個值，那就把nonlocal改成global
        # python 2.7是不支援nonlocal的
        nonlocal x # 引用類屬性的x=200
        print("x=%d" %x) # 局部變量的x，是指向類屬性的x，所以印出x=200
        x = 100 # 因爲這層的x已經指向到了類屬性的x，所以當對局部x做修改成100，那類屬性的x也變成了100
        print("x=%d" %x) # 印出類屬性x的值，x=100
    return test2


x = 300
t1 = test1()
t1() # 分別印出「x=200」「x=100」
print(x) # 在「print(x)」這層找得到x，所以印出這層看到的x值300
# 當第二次調用t1時，因爲類屬性x已經在第一次調用時，被修改成100了，所以印出的就都是100
t1() # 分別印出「x=100」「x=100」，所以代表在test2()中呼叫nonlocal，是真實的去nonlocal的記憶體修改其值


In [ ]:
《裝飾器》
https://www.udemy.com/course/python-en/learn/lecture/15690866#overview

# 裝飾器的功能在於，能夠在不改動函數源碼的情況下，去對函數的功能進行修改


def foo(x):
    print(x)

foo = lambda x: x + 1

foo(3) # 執行結果是4
# 本來foo是個變量名，指向函數foo的記憶體位置，但後來被修改成，指向到(匿名)函數lambda的記憶體位置


def set_func(func):
    def call_func():
        print("--權限驗證1--")
        print("--權限驗證2--")
        func()
    return call_func


def test1():
    print("--test1--")


# ret是return的縮寫
# ret會指向call_func所指向
ret = set_func(test1) 
ret


# 既然變量名可以用ret，那也可以用原函數名test1去取代。那test1會指向閉包內層函數的記憶體位置
# 因爲使用了原函數的名稱test1，所以原本調用test1()的地方，可以繼續沿用test1()的調用，但實際執行的code會變成是執行閉包的內層函數
# 也就是說，要在閉包內層，有呼叫原test1()的函數才算是對test1()做了修改
# 這樣對的code修改方式，就同於在test1()的前面或後面多增加一些code，但不會改動到原型test1()內部的code
# 所以也稱爲「修飾器」，代表不是把原code做修改，而是在上面錦上添花一些東西，修飾而已
test1 = set_func(test1) 
test1()

-----------

@set_func
def test1():
    print("--test1--")

# 掛了@set_func裝飾器在test1()上面，就等價於寫了「test1 = set_func(test1)」
# 也就是會先去調用裝飾器的內層函數，然後內層函數裏面，會有code去調用原test1()函數
test1()


In [ ]:
《裝飾器 - 有參數，但無返回值》

# func就是準備要被修飾的函數
def set_func(func):
    def call_func(a):
        print("--權限驗證1--")
        print("--權限驗證2--")
        func(a) # 未修飾前的函數
    return call_func

# compiler一看到@，就知道要跳先去set_func()，把下面的test1當實參傳進去，變成set_func(test1)
# 然後讓test1指向set_func()內層函數所在的位置(地址)
@set_func 
# 原本的code有一堆test1，阿我就是想把那些有test1的地方加些功能，直觀想是，那就直接在test1內碼改就好
# 但因爲不是全部有調用的地方都要加，所以如果可以在要加的地方，在外面加個@符號就好，那豈不簡單多了
# @set_func就是 xx = set_func(test1)，讓原本調用test1的地方，增加一些功能，並把那些功能寫在內層函數裏面，xx會指向set_func()的內層函數地址
# 但把有調用test1的地方的code，改成xx()多麻煩，所以直接把變量名xx繼續命名成test1就好了
def test1(num):
    print("--test1--%d" % num)


# test1指向set_func內層函數地址，所以test1(100)是調用set_func內層函數，並傳入實參100
# 實參100，傳給裝飾器內層def的形參a
# def的形參a，會傳給def內的程式碼func(a)，func(a)會拿着a去呼叫def test1(num)的code，把實參a代入形參num
# func這個形參的實參就是test1,所以結果就是呼叫test1(100)的原碼
test1(100)



In [ ]:
《裝飾器 - 步驟拆解》

# 1. set_func指向一個新創建的記憶體空間，裝了參數名func
def set_func(func): 
    print("--開始進行裝飾--")
    # 3. 在set_func的空間下，創建了一個空間call_func_1，裝了形參a
    # 3. 因爲看得到呼叫的外層變量，所以知道func指向實參test1
    # 7. 在set_func的空間下，創建了另一個空間call_func_2，裝了形參a
    # 7. 因爲看得到呼叫的外層變量，所以知道func指向實參test2
    # 7. 此時，set_func空間下，有2個call_func的子空間
    def call_func(a):
        print("--權限驗證1--")
        func(a)
    # 4. 返回call_func_1的地址給變量test1
    # 8. 返回call_func_2的地址給變量test2
    return call_func


# 2. 相當於執行 test1 = set_func(test1)
# 2. 調用set_func()，讓形參func指向實參test1
# 5. 變量test1指向call_func_1的地址
@set_func # 4. 一旦看到@，就會執行outter，然後才會讓orifunc指向inner，但還不會執行inner
def test1(num):
    print("--test1--%d" % num)


# 第1次print出來「--開始進行裝飾--」
# 所以compiler一看到@符號，就會開始執行@那行命令去裝飾了，而不是等到函數被呼叫的時候才做


# 6. 執行 test2 = set_func(test2)
# 6. 調用set_func()，讓形參func指向實參test2
# 9. 變量test2指向call_func_2的地址
@set_func
def test2(num):
    print("--test2--%d" % num)


# 10. 裝飾2個函數，就是創建2個內層函數空間
# 10. 並且讓原函數名，轉指向那個新創建的內層函數空間
# 第2次print出來「--開始進行裝飾--」
# 因為這邊都只有做def，而沒有調用，所以並不會print出「--權限驗證1--」、「--權限驗證2--」


In [ ]:
《不定參數1》

def test1(num, *args, **kwargs):
    print("--test1--%d" % num)
    print("--test2--", args)
    print("--test3--", kwargs)

test1(100)
test1(100, 200)
test1(100, 200, 300, mm=100) # 關鍵字參數/命名參數

"""
--test1--100
--test2-- ()
--test3-- {}
--test1--100
--test2-- (200,)
--test3-- {}
--test1--100
--test2-- (200, 300)
--test3-- {'mm': 100}
"""

In [ ]:
《不定參數2》

def test1(num, *args, **kwargs):
    print("--test1--%d" % num)
    # 因爲「% args」，所以代表後面的args有幾個值，前面的"..."裏面就要有幾個%去搭配
    # 而100指派給了num，所以args是空的，但"..."裏面卻有1個%d，所以不匹配
    print("--test2--%d" % args) # TypeError: not enough arguments for format string
    
test1(100)


In [ ]:
《不定參數3》

def test1(num, *args, **kwargs):
    print("--test1--%d" % num)
    print("--test2--%d" % args)
    print("--test3--", kwargs)


test1(100, 200)
# 爲了讓args不爲空，所以改成test1(100, 200)之後，就會輸出下面這樣：
# --test1--100
# --test2--200
# --test3-- {}


In [ ]:
《不定參數4》

def test1(num, *args, **kwargs):
    print(args) # (200, 300)
    print("--test1--%d" % num)
    print("--test2--%d" % args) # TypeError: not all arguments converted during string formatting
    # 爲了和後面% args的個數做匹配，讓"..."裏面也出現2個%，那就可以成功輸出
    # print("--test2--%d %d" % args) # --test2--200 300


test1(100, 200, 300)
# 但如果爲了讓args不爲空，所以改成test1(100, 200, 300)的話，就會出現新的錯誤
# 因爲此時args=(200, 300)，也就是%後面有2個值會被丟去前面的"..."做format，但"..."裏面卻只有出現1個%，所以不匹配


In [ ]:
《不定參數 - 拆包》

def test1(num, *args, **kwargs):
    print(args) # (200, 300)
    print(kwargs) # {'mm': 100, 'nn': 200}
    print(*args) # 200 300
    print(**kwargs) # TypeError: 'a' and 'b' are an invalid keyword argument for print()

test1(100, 200, 300, a=100, b =200)

In [ ]:
《裝飾器 - 接收不定參數》

def set_func(func): 
    print("--開始進行裝飾--")
    def call_func(a):
        print("--權限驗證1--")
        func(a)
    return call_func


@set_func # 相當於 test1 = set_func(test1)
def test1(num, *args, **kwargs):
    print("--test1--%d" % num)
    print("--test2--", args)
    print("--test3--", kwargs)


# 1個實參丟過去，那邊內層函數有1個形參接收，ok沒問題
test1(100) 


# 內層函數只有1個形參，丟2個實參過去會出錯
# 如果把內層函數改成2個形參，那test1(100, 200)雖然不會出錯，但就會變成上面那行test1(100)報錯了
test1(100, 200) 

----------------------

# 所以乾脆把內層函數call_func改成接收不定參數
def set_func(func): 
    print("--開始進行裝飾--")
    # 告訴Python解釋器，如果有多餘的一般參數就都給*args，多餘的關鍵字參數都給**kwargs
    def call_func(*args, **kwargs):
        print("--權限驗證1--")
        # args，是一個元組的包(value1, value2, ...)
        # kwargs，是一個字典的包{key1:value1, key2:value2, ...}
        # 這邊的*args代表對元組拆包，就是把元組拆掉，一個個傳入value
        # 這邊的**kwargs代表對字典拆包，就是把字典外層拆掉，一個個傳入key=value
        func(*args, **kwargs) 
    return call_func)


# 因爲不知道要從被裝飾的函數中，接收幾個參數進來
# 而即使沒有參數，(*args, **kwargs)也不會報錯，所以裝飾器內層函數常用這樣的不定參數表示
# 同理，裝飾器內要呼叫原函數的時候，要原樣的把接收進來的參數，傳出去給原函數
# 因爲傳進來內層函數的時候，由於實參可能形態可能有一般參數和關鍵字參數2種，所以分別會打包成元組和字典2種形態
# 所以要原樣傳出去給原函數，那就代表要用*和**去做解包
# 如果傳給原函數的是(args, kwargs)，那就代表傳遞了2個實參，一個元組，一個字典
# 比如帶有@set_func裝飾器的test1(11,22,33)，在call_func()內寫成func(args, kwargs)的話，那就會是func((11,22,33))
# 寫成func(*args, **kwargs)，才會是func(11,22,33)



@set_func # 相當於 test1 = set_func(test1)
def test1(num, *args, **kwargs):
    print("--test1--%d" % num)
    print("--test2--", args)
    print("--test3--", kwargs)


# (100, 200, 300, mm=100)傳給內層函數call_func(*args, **kwargs)
# args = (100, 200, 300)；kwargs = {mm=100}
# 執行完print("--權限驗證1--")之後，會呼叫func(*args, **kwargs)
# 一般函數中的*和**，是去對實參做解包，所以會還原成(100, 200, 300, mm=100)
# 因爲func指向的是test1的位置，所以func(*args, **kwargs)這行，其實就是執行test1(100, 200, 300, mm=100)
test1(100, 200, 300, mm=100)


In [ ]:
《裝飾器 - 裝飾器有return》

def set_func(func): 
    def call_func(a):
        print("--權限驗證1--")
        # 3. 先做完func(a)，所以印出「--test1--100」
        # 4. 這邊沒有變數來接收func(a)的return，那指向"ok"的指標就會遺失
        # 5. 雖然這邊沒有建一個變數去接收，但上一層呼叫call_func()的人，有變數ret來做接收，保存了指向"ok"的指標地址
        # 6. 所以將func(a)的返回值「"ok"」，返回給test1(100)        
        return func(a) 
    return call_func


# 1. 一旦看到@，就會執行outter，然後才會讓orifunc指向inner，但還不會執行inner
# 2. 相當於 test1 = set_func(test1)
@set_func 
def test1(num, *args, **kwargs):
    print("--test1--%d" % num)
    return "ok" # 返回給呼叫的func(a)


# 7. ret會指向「"ok"」
# 8. 如果是ret = test1，那ret就是指向函數的地址，所以會print出一串記憶體地址
# 9. 但test1(100)是調用，所以如果內層函數call_func()沒有return值，那就會返回None，而print出None
ret = test1(100)
# --權限驗證1--
# --test1--100
print(ret) # ok



In [ ]:
《裝飾器 - 被裝飾的函數，沒有return》

def set_func(func): 
    def call_func(a):
        print("--權限驗證1--")
        # 1. 被裝飾的函數沒有return，所以default會返回None
        # 2. func(a)得到返回值None
        # 3. 將None返回給test2(100)
        # 6. 所以即使被裝飾的函數沒有return，在裝飾器的內層函數仍然可以寫return，不會報錯
        # 7. 但內層函數有return時，可以提升相容性，也就是不管被裝飾的函數有或沒有return，都可以應付
        return func(a) 
    return call_func


@set_func # 相當於 test2 = set_func(test2)
def test2(num, *args, **kwargs):
    print("--test2--%d" % num)


# 4. ret指向None
ret = test2(100)
# --權限驗證1--
# --test2--100
print(ret) # 5. None


In [ ]:
《裝飾器 - 通用裝飾器》

# 也就是說，通用裝飾器是：
def 裝飾器名(func): 
    類屬性值
    def 內層函數(*args, **kwargs):
        物件屬性值
        增加的code
        xxx = 被裝飾函數(*args, **kwargs)
        增加的code
        return xxx
    return 內層函數

In [ ]:
《裝飾器 - 一函數多個裝飾器1》

def set_func_1(func): 
    print("--開始裝飾1--")
    def call_func(*args, **kwargs):
        print("--權限驗證1--")
        return func(*args, **kwargs)
    return call_func


def set_func_2(func): 
    print("--開始裝飾2--")
    def call_func(*args, **kwargs):
        print("--權限驗證2--")
        return func(*args, **kwargs)
    return call_func


@set_func_1
@set_func_2
def test1():
    print("--test1--")


test1()
# --開始裝飾2--
# --開始裝飾1--
# --權限驗證1--
# --權限驗證2--
# --test1--

In [ ]:
《裝飾器 - 一函數多個裝飾器2》

# 因爲要執行「函數名 = 裝飾器名(函數名)」，所以裝飾器下面必須是函數
# 但@decrator_1下面是@decrator_2，沒辦法先做啊
# 所以就讓最接近函數的@decrator_2先去對orifunc裝飾，因爲函數裝飾完就還是一個函數
# 這時候@decrator_1的下面，就是個被@decrator_2裝飾過的函數了
# 所以@decrator_1就可以去對，已經裝飾過的函數，再裝飾一層，變成2層裝飾的「orifunc函數」
# 結論，「裝飾器是拿來裝飾函數，不是拿來裝飾裝飾器」，所以要從最接近函數的地方先做，然後再一步步拉遠
# 所以先印「--開始裝飾2--」，才印「--開始裝飾1--」


# 1. 變數decrator_1，指向新創建的記憶體空間
def decrator_1(func): 
    # 13. 形參func指向實參decrator2的inner位置，print出「--開始裝飾1--」
    print("--開始裝飾1--")
    # 14. 創建decrator1的閉包空間inner（決定了decrator1要對decrator2的inner做裝飾）
    def inner(*args, **kwargs):
        print("--權限驗證1--")
        # 17. 形參func指向的是decrator2的inner，所以進入decrator2執行inner()
        # 18. 然後將decrator2中inner的return值，返回做為「orifunc()」的調用結果值
        return func(*args, **kwargs) # 26. 返回None
    # 15. decrator_1回傳inner的位置，讓orifunc，指向的是decrator1的inner的位置
    return inner


# 2. 變數decrator_2，指向新創建的記憶體空間
def decrator_2(func): 
    # 7. 讓形參func指向實參orifunc的位置，print出「--開始裝飾2」
    print("--開始裝飾2--")
    # 8. 創建decrator2的閉包空間inner（決定了decrator2要對orifunc做裝飾）
    def inner(*args, **kwargs):
        # 19. print出--權限驗證2--
        print("--權限驗證2--")
        # 20. decrator2的inner，其內func指向的是實參orifunc的位置
        # 21. func（也就是orifunc）的調用結果，要return回去給decrator1的inner，當做return值
        # 22. 所以要先進入到實參orifunc
        # 24. 因爲實參orifunc沒有回傳值，所以一路return None回去
        return func(*args, **kwargs) # 25. 返回None
    # 9. decrator_2回傳innder的位置
    return inner


# 11. 裝飾器要包裝的是函數orifunc
# 12. 只不過，現在orifunc已經改指向到decrator2的inner的位置
# 16. 對函數orifunc的所有裝飾器指令都結束了，所以跳到orifunc()那行
@decrator_1 # 3. 下面不是函數，所以保留，先跳進下一行
@decrator_2 # 4. 下面是函數，所以開始進行裝飾
# 5. 變數orifunc，指向新創建的記憶體空間
# 6. 執行裝飾器，等價於執行orifunc = decrator_2(orifunc)
# 10. 「變數orifunc指向decrator2的inner的位置」，結束@decrator_2裝飾器指令，跳回保留的那行@decrator_1指令
def orifunc():
    # 23. print出「--orifunc--」
    print("--orifunc--")


# 16. 因為經過2層裝飾後，orifunc指向的是decrator1的inner的位置，所以print出「--權限驗證1--」
# 27. 沒人接受decrator_1的inner()返回值None，所以None值指標遺失
orifunc()
# --開始裝飾2--
# --開始裝飾1--
# --權限驗證1--
# --權限驗證2--
# --orifunc--


# 因為，裝飾器下面接的，必須是函數才能執行
# 所以，@decrator執行，會是從最接近的orifunc函數開始，由下到上
# --開始裝飾2--
# --開始裝飾1--


# 因為，經過2層裝飾後，orifunc指向的是最上層的inner
# 所以，orifunc()調用，會是從最遠離的orifunc開始，由上到下
# 調用decrator_1的閉包inner()，調用decrator_2的閉包inner()，調用orifunc()
# --權限驗證1--
# --權限驗證2--
# --orifunc--


# 所以，簡單來說，outter執行是由下至上，inner執行是由上至下


In [ ]:
《裝飾器 - demo》

# 簡單來說，outter執行是由下至上，inner執行是由上至下
# 而這邊outter沒有內容，所以看inner部分的執行就好
# 所以執行順序是：td的inner() -> h1的inner() -> haha()
# 又因為td、h1的inner，除了retun上一層的函數外，沒有其他內容
# 所以就是，先得到haha()的return值("haha")
# 然後回傳給h1的inner()當return值(前後包h1)，再回傳給td的inner()當return值(前後再包td)
# 最後再把值繼續回傳，給到了「print(haha()) 」
# 雖然return值沒有變數接收，但是有print做顯示，所以印出了「td><h1>haha</h1></td>」


def decrator_td(func): 
    def inner():
        return "<td>" + func() + "</td>"
    return inner


def decrator_h1(func): 
    def inner():
        return "<h1>" + func() + "</h1>"
    return inner



# 4. 調用"decrator_td"，所以會一路執行到創建decrator_td的閉包空間inner
# 5. 現在haha已經改指向到decrator_h1的inner的位置，不再是原始的位置了
# 6. 然後return會讓變數haha，從原本指向hahadecrator_h1的inner，轉指向到decrator_td的inner的位置
@decrator_td
# 2. 調用"decrator_h1"，所以會一路執行到創建decrator_h1的閉包空間inner
# 3. 然後return會讓變數haha，從原本指向haha原函數，轉指向到decrator_h1的inner的位置
@decrator_h1 # 1. @decrator執行，從最接近的haha()的這邊開始
def haha():
    return "haha"


# haha()調用，從最遠離的haha開始，由上到下，調用decrator_td的閉包inner()，調用decrator_h1的閉包inner()，直到haha()
# decrator_td的inner()要return "<td>" + func() + "</td>"，必須先調用decrator_h1的inner
# decrator_h1的inner()要return "<h1>" + func() + "</h1>"，必須先調用haha()
# haha()中的return值是"haha"。到達底端，開始一步步上一層回傳
# decrator_h1的inner()得到回傳值 "<h1>haha</h1>"
# decrator_td的inner()得到回傳值 "<td><h1>haha</h1></td>"
# 因爲haha指向decrator_td的inner地址
# 所以haha()指向decrator_td的inner的回傳值 "<td><h1>haha</h1></td>"
print(haha()) 
# <td><h1>haha</h1></td>


In [ ]:
《繼承、類方法、類屬性、實例方法、靜態方法》

# Python3中，繼承有沒有寫object沒差
但在Python2的話，有寫就是繼承object，新式類。沒寫就是經典類（經典=舊式）。
__new__會去跟記憶體要一塊位置來創建一個物件，類似創建一個空實例，只有在特殊情況才用


# 類方法，首參數慣用cls，讓這個類的所有物件都改變
在類裡面，def外面，那就是類屬性
每個實例中，都會有一個__class__屬性，用來記錄創建本身實例是哪個類對象，
也就是說，實例可以藉由__class__屬性，去指向找到類屬性的值。
要修改類屬性就用類方法去修改，也就是帶有@classmethod的裝飾器
掛上@classmethod裝飾器後，就算參數不是取cls，默認的第一個參數仍然是指向類對象，
所以既然掛上@classmethod裝飾器，默認會指向類對象，
那麼在@classmethod中做修改的話，就會是去改動到真正的類屬性了。


# 修改類屬性偏門方法是：  物件名.__class__.屬性名 = 'xxx'
比如硬是要由實例去修改類屬性，obj1.country = '美國'，
那只會變成這個實例多了一個實例屬性，country = '美國'，
原類屬性會繼續不變，__class__找到的country仍然是'台灣'。
不過呢，呼叫屬性名的時候，是在實例中找不到，
才會去__class__找看看類有沒有這個屬性名，
那既然自己多加入一個實例屬性，country = '美國'，
自然就不會去__class__找到country = '台灣'。
也就是說，其他實例仍然保持會去__class__找到country = '台灣'，
就只有這個實例會在自己的實例屬性中，就馬上找到country這個屬性名。 


# 物件方法，首參數慣用self，只對個別物件做改變
self的意思是，是前面那個實例，呼叫這個方法
也就是，這個方法作用後的結果，只影響了呼叫的那個實例而已，不影響別的實例
實例方法中的self，就算不取self這個參數名，默認的第一個參數都是指向呼叫的自身實例


def __init__(self, params):
    # 先call父類的__init__()方法當做基礎
    # 有了基本code當base後，才寫自己的code去覆蓋原父類的設定
    super().__init__()


# 靜態方法，沒有默認的首參數，也不是對個別或所有物件改變
掛有@staticmethod裝飾器的函數，其實可以把那個@staticmethod給拿掉，
變成直接在class Foo(object)下的，一個普通函數。
就只是普普通通一個函數，但函數的作用和這個類會有關，
那就放在這個類中，掛個@staticmetho當識別。

In [ ]:
《使用class當作裝飾器》

class Test(object):
    # 3. 物件初始化有給實參orifunc，所以魔法方法這邊也要有形參去接收 
    # 4. 既然是物件，所以有self
    # 拿掉@staticmethod後，其實功能也一樣，就只是怕和別的命名空間重名，所以寫在class裡面做區別的方法
    def __init__(self, func): 
        # 10. 因爲函數名是每個物件不一樣，所以是物件屬性，而不是類屬性 
        # 新建物件Test(orifunc)，實參orifunc會傳給形參func
        # self.(新增/覆寫)屬性func = 賦值orifunc
        self.func = func 
        # 新建物件會調用__init__()，調用完後，形參變量func就會消失
        # 屬性self.func存在於物件orifunc之中，所以只要物件存在，儲存orifunc變量的屬性func就會存在


    # 15. 去調用__call__()的時候，()內也必須要有相對應的(*args, **kwargs)形參去接收才對
    def __call__(self, *args, **kwargs):
        print("調用__call__方法")
        # 7. 我們做@Test就只是個裝飾，原本的orifunc方法是一定要去呼叫執行的
        # 8. 要呼叫物件名orifunc，那就代表類裏面要儲存這個外部來的函數名
        # 9. 要儲存變量，那就是要定義屬性，去接收外部值
        # 11. 解決了儲存這個外部來的函數名之後，所以就可以回到「調用外部原函數名」，這個主題上了
        # 12. func()可能會有返回值， 所以這邊或之後，總要有個地方去接收這個返回值，看是儲存起來，還是print出來
        # 13. main code打算在外面print出來，所以沒有必要在這邊做儲存，直接原封不動的return給呼叫__call__()的位置就好了
        # 14. 裝飾器內要呼叫原函數的時候，要原樣的把接收進來的參數，傳出去給原函數
        # 因爲傳進來內層函數的時候，由於實參可能形態可能有一般參數和關鍵字參數2種，所以分別會打包成元組和字典2種形態
        # 所以要原樣傳出去給原函數，那就代表要用*和**去做解包
        # 如果傳給原函數的是(args, kwargs)，那就代表傳遞了2個實參，一個元組，一個字典
        return self.func(*args, **kwargs)



# 1. orifunc = Test(orifunc)，看起來就像"物件名 = 類名(初始化參數)"
# 2. 所以使用類名當裝飾器的話，看起來就等同是"創建實例"
# 5. 這時orifunc指向的不是函數，而是創建出來的實例對象
@Test 
def orifunc():
    return "--orifunc--"


# 6. orifunc()就變成等價於"實例()"，所以會去執行這個物件類的__call__()
print(orifunc())


In [ ]:
《裝飾器 - 各種概念復習》

# 函數是只有功能，但沒有儲存數據
# 閉包是又有功能，又有儲存數據（實參傳給外層def）
# 物件有功能，也有數據，但有太多不會用到的魔法方法，浪費空間


說到儲存數據這件事，雖然函數也可以到儲存在外層的全局變量去找，
但在Python中摸得到，也就代表可以改得了，
A函數使用的數據，要到外層的全局變量去找，那B函數要用到外層的全局變量時，可能就會改到A用到的變數了。
即使可以通過列表切割數據等方式去防止修改，但要多加處理的步驟就是比較花時間。
而閉包儲存的數據方式，則像是物件屬性一樣，可以做到個人獨有，又不浪費空間。
閉包就是單獨封裝了一個空間，既有函數，又有數據。


# orifunc指向inner，所以orifunc(...)、inner(*args, **kwargs)兩者的參數要一致
# inner的func指向orifunc，所以func(*args, **kwargs)、orifunc(...)兩者的參數要一致
# orifunc中要是有return xxx，inner也要有return func(*args, **kwargs)，把最底層的返回值，返回給最上層的orifunc()呼叫方
# 反正上層就算寫了return沒用到，比如最底層沒有return值，那頂多就是一路返回None去上層
# 上面某層如果沒有變數去接收到，或繼續做返回，那指向值的地址就沒儲存到，會消失掉。也就是值雖然還在記憶體，但卻撈不到了


# @decrator執行裝飾，是從最接近的orifunc函數開始，從內到外。就像打包寄送包裹一樣，「從內而外一層層包裝」
# orifunc()調用，從最遠離的orifunc開始，由外到內。就像要使用寄送來的包裹一樣，「從外而內一層層拆開」
# 調用decrator_1的閉包inner()，調用decrator_2的閉包inner()，直到orifunc()


In [ ]:
《裝飾器 - 通用裝飾器》

def decorator(func):
    def inner(*args, **kwargs):
        return func(*args, **kwargs)
    return inner

@decorator
def orifunc():
    pass

orifunc()


In [ ]:
《裝飾器 - 有參數的裝飾器，3層def》

# 裝飾器有參數的情況下，是讓orifunc轉指向decorator_param(param)的內層函數，也就是decorator(func)
# 這時候decorator(func)外的東西，就是傳給裝飾器的參數(比如權限等級)，而func是要被裝飾的函數
# 所以，閉包由內而外是，「物件屬性 -> 類屬性 -> 裝飾器參數」
# 有一種，「越內層的閉包，存放的資訊越私密」的感覺


# 3. decorator是個有2層def的閉包，並且存有數據param指向arg
# 4. 存有數據的閉包，概念上可以想像成是類屬性，也就是cls.param = arg
# 5. 真正調用的還是inner()，func是儲存要被裝飾的函數，param是儲存裝飾器參數
def decorator_param(param) # param，儲存裝飾器參數
    def decorator(func): # func，儲存要被裝飾的函數
        def inner(*args, **kwargs): # inner()，儲存真正要調用的code
            return func(*args, **kwargs)
        return inner
    return decorator


@decorator_param(arg)
def orifunc():
    pass

# parameter是形參，在函數定義時放在小括號裡
# argument 是實參，在調用函數時放在小括號裡
# 1. 調用decorator_param(arg)的返回值是decorator
# 2. 這時的返回值，就會是熟悉的無參數裝飾器decorator，可以像從前那樣一般的去對orifunc裝飾
orifunc()


In [ ]:
《3層def - 步驟》

# 裝飾器無參數的情況下，是讓orifunc轉指向decorator(func)的內層函數，也就是inner(*args, **kwargs)
# 這時候inner()外的東西就像類屬性，inner()內的東西像物件屬性
# 其實2層def的無參數裝飾器，也和3層def的有參數裝飾器一樣概念
# 2層def，掛上@decorator的orifunc
# 調用decorator()，並將orifunc當作實參傳入
# orifunc指向inner，並且儲存了func指向orifunc的資訊


def decorator(func): # func，儲存要被裝飾的函數
    def inner(*args, **kwargs): # inner()，儲存真正要調用的code
        return func(*args, **kwargs)
    return inner

@decorator
def orifunc():
    pass

orifunc()


# 3層def例如，掛上@decorator_param(arg)的orifunc，是一樣的概念
# 調用decorator_parm()，並將arg當作實參傳入，orifunc指向decorator，並且儲存了param指向arg的資訊
# 換句話說，調用裝飾器所要傳入的實參有2種可能：如果裝飾器有帶參數(3層def)那就傳裝飾器的參數，如果裝飾器沒有帶參數(2層def)那就傳orifunc進去
# 同理，而orifunc的指向，都是裝飾器的第2層def：如果有帶參數(3層def)那orifunc就指向decorator，如果裝飾器沒有帶參數(2層def)那就指向inner


def decorator_param(param) # param，儲存裝飾器參數
    def decorator(func): # func，儲存要被裝飾的函數
        def inner(*args, **kwargs): # inner()，儲存真正要調用的code
            return func(*args, **kwargs)
        return inner
    return decorator

# 1. 看到@就會先執行裝飾，如果有多個，就是從下到上裝飾包裝，然後再從上到下調用inner解開
# 2. 因爲看到@帶有參數arg，所以先裝飾arg
# 3. 裝飾arg就是，在函數decorator_param的空間下，創建一小塊decorator空間，儲存param指向arg，然後回傳這個decorator的記憶體地址
# 4. decorator有2層def，所以仍然是個裝飾器，也就是變成了在def orifunc()上，有一個無參數的裝飾器decorator
# 5. 只是這個裝飾器decorator指向的記憶體位置，裏面已經儲存了儲存param指向arg
# 6. 既然def orifunc()上還是有裝飾器，那就還是要執行裝飾，這時候的步驟就等同2層def的裝飾器了
# 所以傳入orifunc當作實參到decorator(func)，也就是在本來就存有實參arg的decorator中，又多了一個orifunc的實參進去
# 然後回傳值是inner的地址，此時的inner，是在那個存有實參arg、orifunc的decorator中，再新創建出來的一小塊記憶體
# 也就是，decorator_param函數是一大塊記憶體(class)，而閉包的好處就是類似物件實例可以存有多組不同實參
# 而目前這個案例就是，在decorator_param內有一塊區域A(decorator)存有param -> arg，在A內又有一塊區域B(inner)存有func -> orifunc
# 當在B要找param找不到時，就會往上一層A找，由此來形成，將實參打包成分層屬性的現象
# 8. 所以進行2層def裝飾器@decorator對def orifunc()的裝飾，使得orifunc指向inner
# 9. 並從inner的內至外層，分層存有實參func -> orifunc，以及param -> arg
@decorator_param(arg)
def orifunc():
    pass

# 5. 調用orifunc()，就等同調用decorator()，並且預存有param->arg
orifunc()


In [ ]:
《裝飾器 - 有參數的裝飾器_demo》

# 想像不同的函數，因爲其功能深入性不同，所以驗證級別不一樣
# 所以用裝飾器參數去判斷驗證級別，給予不同的條件code


def decorator_param(param):
    def decorator(func):
        def inner(*args, **kwargs): 
            if param == 1: # 把原本的args[0]，改成param
                print("---實參=1---")
            elif param == 2:
                print("---實參=2---")
            return func()
        return inner
    return decorator


# 寫API的人，來決定這個函數的調用權限等級
# 而不應該是像之前用orifunc(1)，讓調用API的人自己決定自己的權限等級
# orifunc(1)就是，明明orifunc在def內沒有定義參數，但加了裝飾器後，卻可以使用參數去調用，這樣雖然可以work，卻不合理
@decorator_param(1)
def orifunc_1():
    print("---orifunc_1---")


@decorator_param(2)
def orifunc_2():
    print("---orifunc_2---")


orifunc_1() 
# ---實參=1---
# ---orifunc_1---
orifunc_2() 
# ---實參=2---
# ---orifunc_2---


In [ ]:
《裝飾器 - orifunc在def內沒有定義參數，加了裝飾器，卻可以使用參數去調用》


# 裝飾後的orifunc會指向inner，所以只要inner有寫了形參，那orifunc傳入實參是也可以執行沒錯
# 不過要注意的是，inner呼叫func()時，參數仍然要給原始正確的個數與形態，否則會報錯
# 但是，orifunc本身定義時，如果是沒有寫參數的，調用就不應該有才對啊！否則之前寫的orifunc(...)調用，就都會報錯


def decorator(func):
    def inner(*args, **kwargs): 
        # args會是一個元組，所以args[0]就是取元組的第0個值
        if args[0] == 1:
            print("---實參=1---")
        elif args[0] == 2:
            print("---實參=2---")
        # 這邊寫通用的func(*args, **kwargs)會報錯 "orifunc() takes 0 positional arguments but 1 was given"
        # 因爲orifunc()和func()的參數，已經不一致
        # orifunc給1個參數，所以inner接收的就是orifunc給1個參數，但func()需要的是0個參數
        return func()
    return inner


@decorator
def orifunc():
    print("---orifunc---")


orifunc(1) 
# ---實參=1---
# ---orifunc---
orifunc(2) 
# ---實參=2---
# ---orifunc---
